In [1]:
# load library

import os, glob
import psycopg2
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import datetime as dt
import ibis
import ibis.selectors as s
from ibis.interactive import *

In [2]:
# options 

ibis.options.interactive = True

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

plt.rcParams['axes.unicode_minus'] = False
mpl.rcParams['font.family'] = "AppleGothic"  # 윈도우는 'Malgun Gothic' 사용하세요. 

In [3]:
# 샘플 데이터 가져오기

ex.penguins

t = ex.penguins.fetch()
t

penguins(name='penguins', help='Size measurements for adult foraging penguins near Palmer Station, Antarctica')

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year  ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ string  │ string    │ float64        │ float64       │ int64             │ int64       │ string │ int64 │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼───────┤
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │  2007 │
│ Adelie  │ Torgersen │           39.5 │          17.4 │               186 │        3800 │ female │  2007 │
│ Adelie  │ Torgersen │           40.3 │          18.0 │               195 │        3250 │ female │  2007 │
│ Adelie  │ Torgersen │           NULL │          NULL │              NULL │        NULL │ NULL   │  2007 │
│ Adelie  │ Torgersen │           36.7 │          19.3 │               193 │        3450 │ female │  2007 │
│ Adelie  │ Torgersen │           39.3 │          20.6 │               190 │        3650 │ male   │  2007 │
│ Adelie  │ Torgersen │           38.9 │          17.8 │               181 │        3625 │ female │  2007 │
│ Adelie  │ Torgersen │           39.2 │          19.6 │               195 │        4675 │ male   │  2007 │
│ Adelie  │ Torgersen │           34.1 │          18.1 │               193 │        3475 │ NULL   │  2007 │
│ Adelie  │ Torgersen │           42.0 │          20.2 │               190 │        4250 │ NULL   │  2007 │
│ …       │ …         │              … │             … │                 … │           … │ …      │     … │
└─────────┴───────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴───────┘

In [ ]:
# 칼럼별 평균 값 계산 : 비추

t.agg(t[c].mean() for c in t.columns if t[c].type().is_numeric())

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Mean(bill_length_mm) ┃ Mean(bill_depth_mm) ┃ Mean(flipper_length_mm) ┃ Mean(body_mass_g) ┃ Mean(year) ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ float64              │ float64             │ float64                 │ float64           │ float64    │
├──────────────────────┼─────────────────────┼─────────────────────────┼───────────────────┼────────────┤
│             43.92193 │            17.15117 │              200.915205 │       4201.754386 │ 2008.02907 │
└──────────────────────┴─────────────────────┴─────────────────────────┴───────────────────┴────────────┘

In [58]:
# ibis style

# 수치형 변수만 선택해서 평균 구하기
t.select(s.across(s.numeric(), _.mean()))
t.select(s.across(s.numeric(), _.mean())).head(1)

# 각각의 값에서 평균을 빼기
t
t.mutate(s.across(s.numeric(), _ - _.mean()))

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ year       ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ float64        │ float64       │ float64           │ float64     │ float64    │
├────────────────┼───────────────┼───────────────────┼─────────────┼────────────┤
│       43.92193 │      17.15117 │        200.915205 │ 4201.754386 │ 2008.02907 │
│       43.92193 │      17.15117 │        200.915205 │ 4201.754386 │ 2008.02907 │
│       43.92193 │      17.15117 │        200.915205 │ 4201.754386 │ 2008.02907 │
│       43.92193 │      17.15117 │        200.915205 │ 4201.754386 │ 2008.02907 │
│       43.92193 │      17.15117 │        200.915205 │ 4201.754386 │ 2008.02907 │
│       43.92193 │      17.15117 │        200.915205 │ 4201.754386 │ 2008.02907 │
│       43.92193 │      17.15117 │        200.915205 │ 4201.754386 │ 2008.02907 │
│       43.92193 │      17.15117 │        200.915205 │ 4201.754386 │ 2008.02907 │
│       43.92193 │      17.15117 │        200.915205 │ 4201.754386 │ 2008.02907 │
│       43.92193 │      17.15117 │        200.915205 │ 4201.754386 │ 2008.02907 │
│              … │             … │                 … │           … │          … │
└────────────────┴───────────────┴───────────────────┴─────────────┴────────────┘

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ year       ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ float64        │ float64       │ float64           │ float64     │ float64    │
├────────────────┼───────────────┼───────────────────┼─────────────┼────────────┤
│       43.92193 │      17.15117 │        200.915205 │ 4201.754386 │ 2008.02907 │
└────────────────┴───────────────┴───────────────────┴─────────────┴────────────┘

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year  ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ string  │ string    │ float64        │ float64       │ int64             │ int64       │ string │ int64 │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼───────┤
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │  2007 │
│ Adelie  │ Torgersen │           39.5 │          17.4 │               186 │        3800 │ female │  2007 │
│ Adelie  │ Torgersen │           40.3 │          18.0 │               195 │        3250 │ female │  2007 │
│ Adelie  │ Torgersen │           NULL │          NULL │              NULL │        NULL │ NULL   │  2007 │
│ Adelie  │ Torgersen │           36.7 │          19.3 │               193 │        3450 │ female │  2007 │
│ Adelie  │ Torgersen │           39.3 │          20.6 │               190 │        3650 │ male   │  2007 │
│ Adelie  │ Torgersen │           38.9 │          17.8 │               181 │        3625 │ female │  2007 │
│ Adelie  │ Torgersen │           39.2 │          19.6 │               195 │        4675 │ male   │  2007 │
│ Adelie  │ Torgersen │           34.1 │          18.1 │               193 │        3475 │ NULL   │  2007 │
│ Adelie  │ Torgersen │           42.0 │          20.2 │               190 │        4250 │ NULL   │  2007 │
│ …       │ …         │              … │             … │                 … │           … │ …      │     … │
└─────────┴───────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴───────┘

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year     ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩
│ string  │ string    │ float64        │ float64       │ float64           │ float64     │ string │ float64  │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼──────────┤
│ Adelie  │ Torgersen │       -4.82193 │       1.54883 │        -19.915205 │ -451.754386 │ male   │ -1.02907 │
│ Adelie  │ Torgersen │       -4.42193 │       0.24883 │        -14.915205 │ -401.754386 │ female │ -1.02907 │
│ Adelie  │ Torgersen │       -3.62193 │       0.84883 │         -5.915205 │ -951.754386 │ female │ -1.02907 │
│ Adelie  │ Torgersen │           NULL │          NULL │              NULL │        NULL │ NULL   │ -1.02907 │
│ Adelie  │ Torgersen │       -7.22193 │       2.14883 │         -7.915205 │ -751.754386 │ female │ -1.02907 │
│ Adelie  │ Torgersen │       -4.62193 │       3.44883 │        -10.915205 │ -551.754386 │ male   │ -1.02907 │
│ Adelie  │ Torgersen │       -5.02193 │       0.64883 │        -19.915205 │ -576.754386 │ female │ -1.02907 │
│ Adelie  │ Torgersen │       -4.72193 │       2.44883 │         -5.915205 │  473.245614 │ male   │ -1.02907 │
│ Adelie  │ Torgersen │       -9.82193 │       0.94883 │         -7.915205 │ -726.754386 │ NULL   │ -1.02907 │
│ Adelie  │ Torgersen │       -1.92193 │       3.04883 │        -10.915205 │   48.245614 │ NULL   │ -1.02907 │
│ …       │ …         │              … │             … │                 … │           … │ …      │        … │
└─────────┴───────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴──────────┘

In [59]:
# 표준화

t.mutate(s.across(s.numeric(), (_ - _.mean()) / _.std()))

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year      ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━┩
│ string  │ string    │ float64        │ float64       │ float64           │ float64     │ string │ float64   │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼───────────┤
│ Adelie  │ Torgersen │      -0.883137 │      0.784218 │         -1.416243 │   -0.563316 │ male   │ -1.258032 │
│ Adelie  │ Torgersen │      -0.809877 │      0.125990 │         -1.060674 │   -0.500969 │ female │ -1.258032 │
│ Adelie  │ Torgersen │      -0.663357 │      0.429788 │         -0.420652 │   -1.186793 │ female │ -1.258032 │
│ Adelie  │ Torgersen │           NULL │          NULL │              NULL │        NULL │ NULL   │ -1.258032 │
│ Adelie  │ Torgersen │      -1.322698 │      1.088015 │         -0.562879 │   -0.937402 │ female │ -1.258032 │
│ Adelie  │ Torgersen │      -0.846507 │      1.746243 │         -0.776220 │   -0.688012 │ male   │ -1.258032 │
│ Adelie  │ Torgersen │      -0.919767 │      0.328522 │         -1.416243 │   -0.719185 │ female │ -1.258032 │
│ Adelie  │ Torgersen │      -0.864822 │      1.239914 │         -0.420652 │    0.590115 │ male   │ -1.258032 │
│ Adelie  │ Torgersen │      -1.798888 │      0.480420 │         -0.562879 │   -0.906228 │ NULL   │ -1.258032 │
│ Adelie  │ Torgersen │      -0.352002 │      1.543712 │         -0.776220 │    0.060160 │ NULL   │ -1.258032 │
│ …       │ …         │              … │             … │                 … │           … │ …      │         … │
└─────────┴───────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴───────────┘

In [60]:
# 수치형 변수를 표준화 하지만 연도는 제외

t.mutate(s.across(s.numeric() & ~s.c("year"), (_ - _.mean()) / _.std()))

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year  ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ string  │ string    │ float64        │ float64       │ float64           │ float64     │ string │ int64 │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼───────┤
│ Adelie  │ Torgersen │      -0.883137 │      0.784218 │         -1.416243 │   -0.563316 │ male   │  2007 │
│ Adelie  │ Torgersen │      -0.809877 │      0.125990 │         -1.060674 │   -0.500969 │ female │  2007 │
│ Adelie  │ Torgersen │      -0.663357 │      0.429788 │         -0.420652 │   -1.186793 │ female │  2007 │
│ Adelie  │ Torgersen │           NULL │          NULL │              NULL │        NULL │ NULL   │  2007 │
│ Adelie  │ Torgersen │      -1.322698 │      1.088015 │         -0.562879 │   -0.937402 │ female │  2007 │
│ Adelie  │ Torgersen │      -0.846507 │      1.746243 │         -0.776220 │   -0.688012 │ male   │  2007 │
│ Adelie  │ Torgersen │      -0.919767 │      0.328522 │         -1.416243 │   -0.719185 │ female │  2007 │
│ Adelie  │ Torgersen │      -0.864822 │      1.239914 │         -0.420652 │    0.590115 │ male   │  2007 │
│ Adelie  │ Torgersen │      -1.798888 │      0.480420 │         -0.562879 │   -0.906228 │ NULL   │  2007 │
│ Adelie  │ Torgersen │      -0.352002 │      1.543712 │         -0.776220 │    0.060160 │ NULL   │  2007 │
│ …       │ …         │              … │             … │                 … │           … │ …      │     … │
└─────────┴───────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴───────┘

In [61]:
# sql로 보면...

tmp = t.mutate(s.across(s.numeric() & ~s.c("year"), (_ - _.mean()) / _.std()))

ibis.show_sql(tmp)

SELECT
  t0.species,
  t0.island,
  (
    t0.bill_length_mm - AVG(t0.bill_length_mm) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
  ) / CAST(STDDEV_SAMP(t0.bill_length_mm) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS DECIMAL(18, 3)) AS bill_length_mm,
  (
    t0.bill_depth_mm - AVG(t0.bill_depth_mm) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
  ) / CAST(STDDEV_SAMP(t0.bill_depth_mm) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS DECIMAL(18, 3)) AS bill_depth_mm,
  (
    t0.flipper_length_mm - AVG(t0.flipper_length_mm) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
  ) / CAST(STDDEV_SAMP(t0.flipper_length_mm) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS DECIMAL(18, 3)) AS flipper_length_mm,
  (
    t0.body_mass_g - AVG(t0.body_mass_g) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
  ) / CAST(STDDEV_SAMP(t0.body_mass_g) OVER (ROWS BETWEEN UNBOUNDED PREC

In [63]:
# 그룹별 적용

t.group_by("species").mutate(s.across(s.numeric() & ~s.c("year"), (_ - _.mean()) / _.std()))

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year  ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ string  │ string    │ float64        │ float64       │ float64           │ float64     │ string │ int64 │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼───────┤
│ Adelie  │ Torgersen │       0.791817 │     -1.270631 │          0.160018 │   -0.001444 │ female │  2008 │
│ Adelie  │ Biscoe    │       1.467747 │     -0.038092 │          0.924661 │    0.816323 │ male   │  2009 │
│ Adelie  │ Dream     │       0.378749 │      0.619262 │         -0.910482 │    2.070231 │ male   │  2007 │
│ Adelie  │ Dream     │      -0.860455 │     -0.284600 │         -1.216339 │   -1.200835 │ female │  2007 │
│ Adelie  │ Torgersen │      -0.785351 │      0.783601 │          0.465875 │   -0.546622 │ female │  2007 │
│ Adelie  │ Torgersen │       0.190991 │      1.851801 │          0.007089 │   -0.110480 │ male   │  2007 │
│ Adelie  │ Torgersen │       0.040785 │     -0.448938 │         -1.369268 │   -0.164998 │ female │  2007 │
│ Adelie  │ Torgersen │       0.153439 │      1.030109 │          0.771732 │    2.124749 │ male   │  2007 │
│ Adelie  │ Torgersen │      -1.761694 │     -0.202430 │          0.465875 │   -0.492104 │ NULL   │  2007 │
│ Adelie  │ Torgersen │       1.204885 │      1.523124 │          0.007089 │    1.197947 │ NULL   │  2007 │
│ …       │ …         │              … │             … │                 … │           … │ …      │     … │
└─────────┴───────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴───────┘

In [71]:
# 데이터 형 변환

t.mutate(s.across(s.endswith(("_mm", "_g")), _.cast("float32")))

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year  ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ string  │ string    │ float32        │ float32       │ float32           │ float32     │ string │ int64 │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼───────┤
│ Adelie  │ Torgersen │      39.099998 │     18.700001 │             181.0 │      3750.0 │ male   │  2007 │
│ Adelie  │ Torgersen │      39.500000 │     17.400000 │             186.0 │      3800.0 │ female │  2007 │
│ Adelie  │ Torgersen │      40.299999 │     18.000000 │             195.0 │      3250.0 │ female │  2007 │
│ Adelie  │ Torgersen │           NULL │          NULL │              NULL │        NULL │ NULL   │  2007 │
│ Adelie  │ Torgersen │      36.700001 │     19.299999 │             193.0 │      3450.0 │ female │  2007 │
│ Adelie  │ Torgersen │      39.299999 │     20.600000 │             190.0 │      3650.0 │ male   │  2007 │
│ Adelie  │ Torgersen │      38.900002 │     17.799999 │             181.0 │      3625.0 │ female │  2007 │
│ Adelie  │ Torgersen │      39.200001 │     19.600000 │             195.0 │      4675.0 │ male   │  2007 │
│ Adelie  │ Torgersen │      34.099998 │     18.100000 │             193.0 │      3475.0 │ NULL   │  2007 │
│ Adelie  │ Torgersen │      42.000000 │     20.200001 │             190.0 │      4250.0 │ NULL   │  2007 │
│ …       │ …         │              … │             … │                 … │           … │ …      │     … │
└─────────┴───────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴───────┘

In [74]:
# 데이터 값을 소문자로 변경

t.mutate(s.across(s.of_type("string"), _.lower()))

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ year  ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ string  │ string    │ float64        │ float64       │ int64             │ int64       │ string │ int64 │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼───────┤
│ adelie  │ torgersen │           39.1 │          18.7 │               181 │        3750 │ male   │  2007 │
│ adelie  │ torgersen │           39.5 │          17.4 │               186 │        3800 │ female │  2007 │
│ adelie  │ torgersen │           40.3 │          18.0 │               195 │        3250 │ female │  2007 │
│ adelie  │ torgersen │           NULL │          NULL │              NULL │        NULL │ NULL   │  2007 │
│ adelie  │ torgersen │           36.7 │          19.3 │               193 │        3450 │ female │  2007 │
│ adelie  │ torgersen │           39.3 │          20.6 │               190 │        3650 │ male   │  2007 │
│ adelie  │ torgersen │           38.9 │          17.8 │               181 │        3625 │ female │  2007 │
│ adelie  │ torgersen │           39.2 │          19.6 │               195 │        4675 │ male   │  2007 │
│ adelie  │ torgersen │           34.1 │          18.1 │               193 │        3475 │ NULL   │  2007 │
│ adelie  │ torgersen │           42.0 │          20.2 │               190 │        4250 │ NULL   │  2007 │
│ …       │ …         │              … │             … │                 … │           … │ …      │     … │
└─────────┴───────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴───────┘

In [83]:
# 표준화 값들을 추가로 생성하려면

expr = t.mutate(s.across(s.endswith(("_mm", "_g")), dict(centered = _ - _.mean(), zscore = _ - _.mean() / _.std())))
expr.columns

expr.select(s.contains("centered"))
expr.select(s.contains(["centered", "zscore"]))


['species',
 'island',
 'bill_length_mm',
 'bill_depth_mm',
 'flipper_length_mm',
 'body_mass_g',
 'sex',
 'year',
 'bill_length_mm_centered',
 'bill_depth_mm_centered',
 'flipper_length_mm_centered',
 'body_mass_g_centered',
 'bill_length_mm_zscore',
 'bill_depth_mm_zscore',
 'flipper_length_mm_zscore',
 'body_mass_g_zscore']

┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ bill_length_mm_centered ┃ bill_depth_mm_centered ┃ flipper_length_mm_centered ┃ body_mass_g_centered ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ float64                 │ float64                │ float64                    │ float64              │
├─────────────────────────┼────────────────────────┼────────────────────────────┼──────────────────────┤
│                -4.82193 │                1.54883 │                 -19.915205 │          -451.754386 │
│                -4.42193 │                0.24883 │                 -14.915205 │          -401.754386 │
│                -3.62193 │                0.84883 │                  -5.915205 │          -951.754386 │
│                    NULL │                   NULL │                       NULL │                 NULL │
│                -7.22193 │                2.14883 │                  -7.915205 │          -751.754386 │
│                -4.62193 │                3.44883 │                 -10.915205 │          -551.754386 │
│                -5.02193 │                0.64883 │                 -19.915205 │          -576.754386 │
│                -4.72193 │                2.44883 │                  -5.915205 │           473.245614 │
│                -9.82193 │                0.94883 │                  -7.915205 │          -726.754386 │
│                -1.92193 │                3.04883 │                 -10.915205 │            48.245614 │
│                       … │                      … │                          … │                    … │
└─────────────────────────┴────────────────────────┴────────────────────────────┴──────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ bill_length_mm_centered ┃ bill_depth_mm_centered ┃ flipper_length_mm_centered ┃ body_mass_g_centered ┃ bill_length_mm_zscore ┃ bill_depth_mm_zscore ┃ flipper_length_mm_zscore ┃ body_mass_g_zscore ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ float64                 │ float64                │ float64                    │ float64              │ float64               │ float64              │ float64                  │ float64            │
├─────────────────────────┼────────────────────────┼────────────────────────────┼──────────────────────┼───────────────────────┼──────────────────────┼──────────────────────────┼────────────────────┤
│                -4.82193 │                1.54883 │                 -19.915205 │          -451.754386 │             31.055691 │            10.015863 │               166.712189 │        3744.760611 │
│                -4.42193 │                0.24883 │                 -14.915205 │          -401.754386 │             31.455691 │             8.715863 │               171.712189 │        3794.760611 │
│                -3.62193 │                0.84883 │                  -5.915205 │          -951.754386 │             32.255691 │             9.315863 │               180.712189 │        3244.760611 │
│                    NULL │                   NULL │                       NULL │                 NULL │                  NULL │                 NULL │                     NULL │               NULL │
│                -7.22193 │                2.14883 │                  -7.915205 │          -751.754386 │             28.655691 │            10.615863 │               178.712189 │        3444.760611 │
│                -4.62193 │                3.44883 │                 -10.915205 │          -551.754386 │             31.255691 │            11.915863 │               175.712189 │        3644.760611 │
│                -5.02193 │                0.64883 │                 -19.915205 │          -576.754386 │             30.855691 │             9.115863 │               166.712189 │        3619.760611 │
│                -4.72193 │                2.44883 │                  -5.915205 │           473.245614 │             31.155691 │            10.915863 │               180.712189 │        4669.760611 │
│                -9.82193 │                0.94883 │                  -7.915205 │          -726.754386 │             26.055691 │             9.415863 │               178.712189 │        3469.760611 │
│                -1.92193 │                3.04883 │                 -10.915205 │            48.245614 │             33.955691 │            11.515863 │               175.712189 │        4244.760611 │
│                       … │                      … │                          … │                    … │                     … │                    … │                        … │                  … │
└─────────────────────────┴────────────────────────┴────────────────────────────┴──────────────────────┴───────────────────────┴──────────────────────┴──────────────────────────┴────────────────────┘

In [84]:
# 연도별 수치형 데이터 평균 

t.group_by("year").agg(s.across(s.numeric() & ~ s.c("year"), _.mean()))

┏━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ year  ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ int64 │ float64        │ float64       │ float64           │ float64     │
├───────┼────────────────┼───────────────┼───────────────────┼─────────────┤
│  2007 │      43.740367 │     17.427523 │        196.880734 │ 4124.541284 │
│  2009 │      44.452941 │     17.125210 │        202.806723 │ 4210.294118 │
│  2008 │      43.541228 │     16.914035 │        202.798246 │ 4266.666667 │
└───────┴────────────────┴───────────────┴───────────────────┴─────────────┘

In [85]:
# negation 활용

sel = s.numeric() & ~ s.c("year")

t.group_by(~ sel).agg(s.across(sel, _.mean()))

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ species ┃ island    ┃ sex    ┃ year  ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string  │ string    │ string │ int64 │ float64        │ float64       │ float64           │ float64     │
├─────────┼───────────┼────────┼───────┼────────────────┼───────────────┼───────────────────┼─────────────┤
│ Adelie  │ Biscoe    │ male   │  2007 │      39.160000 │     18.300000 │        181.600000 │ 3770.000000 │
│ Gentoo  │ Biscoe    │ male   │  2007 │      49.000000 │     15.364706 │        218.882353 │ 5552.941176 │
│ Adelie  │ Biscoe    │ male   │  2008 │      40.755556 │     19.033333 │        192.555556 │ 4011.111111 │
│ Adelie  │ Torgersen │ female │  2009 │      37.775000 │     17.087500 │        187.250000 │ 3193.750000 │
│ Adelie  │ Torgersen │ male   │  2009 │      40.850000 │     18.987500 │        198.625000 │ 3784.375000 │
│ Gentoo  │ Biscoe    │ male   │  2008 │      48.539130 │     15.704348 │        222.086957 │ 5410.869565 │
│ Adelie  │ Torgersen │ female │  2008 │      36.612500 │     17.400000 │        190.000000 │ 3518.750000 │
│ Adelie  │ Torgersen │ male   │  2008 │      40.925000 │     18.837500 │        193.500000 │ 4193.750000 │
│ Adelie  │ Biscoe    │ male   │  2009 │      41.300000 │     19.500000 │        193.500000 │ 4268.750000 │
│ Gentoo  │ Biscoe    │ male   │  2009 │      50.880952 │     16.019048 │        223.095238 │ 5510.714286 │
│ …       │ …         │ …      │     … │              … │             … │                 … │           … │
└─────────┴───────────┴────────┴───────┴────────────────┴───────────────┴───────────────────┴─────────────┘

In [95]:
# 값의 범위 벗어나는 값 찾기

expr = t.drop("year")\
        .group_by("species")\
        .mutate(s.across(s.numeric(), dict(zscore = (_ - _.mean())/_.std())))\
        .filter(s.if_all(s.startswith("bill") & s.endswith("_zscore"), _.abs() > 2))

expr

┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ species ┃ island    ┃ bill_length_mm ┃ bill_depth_mm ┃ flipper_length_mm ┃ body_mass_g ┃ sex    ┃ bill_length_mm_zscore ┃ bill_depth_mm_zscore ┃ flipper_length_mm_zscore ┃ body_mass_g_zscore ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ string  │ string    │ float64        │ float64       │ int64             │ int64       │ string │ float64               │ float64              │ float64                  │ float64            │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼────────┼───────────────────────┼──────────────────────┼──────────────────────────┼────────────────────┤
│ Gentoo  │ Biscoe    │           59.6 │          17.0 │               230 │        6050 │ male   │              3.924439 │             2.056969 │                 1.975792 │           1.932063 │
│ Gentoo  │ Biscoe    │           55.9 │          17.0 │               228 │        5600 │ male   │              2.723920 │             2.056969 │                 1.667388 │           1.039411 │
│ Adelie  │ Torgersen │           46.0 │          21.5 │               194 │        4200 │ male   │              2.706951 │             2.591325 │                 0.618804 │           1.088911 │
│ Adelie  │ Dream     │           32.1 │          15.5 │               188 │        3050 │ female │             -2.512727 │            -2.338831 │                -0.298768 │          -1.418906 │
└─────────┴───────────┴────────────────┴───────────────┴───────────────────┴─────────────┴────────┴───────────────────────┴──────────────────────┴──────────────────────────┴────────────────────┘

In [96]:
ibis.show_sql(expr)

WITH t0 AS (
  SELECT
    t2.species AS species,
    t2.island AS island,
    t2.bill_length_mm AS bill_length_mm,
    t2.bill_depth_mm AS bill_depth_mm,
    t2.flipper_length_mm AS flipper_length_mm,
    t2.body_mass_g AS body_mass_g,
    t2.sex AS sex
  FROM main.penguins AS t2
), t1 AS (
  SELECT
    t0.species AS species,
    t0.island AS island,
    t0.bill_length_mm AS bill_length_mm,
    t0.bill_depth_mm AS bill_depth_mm,
    t0.flipper_length_mm AS flipper_length_mm,
    t0.body_mass_g AS body_mass_g,
    t0.sex AS sex,
    (
      t0.bill_length_mm - AVG(t0.bill_length_mm) OVER (PARTITION BY t0.species ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
    ) / CAST(STDDEV_SAMP(t0.bill_length_mm) OVER (PARTITION BY t0.species ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS DECIMAL(18, 3)) AS bill_length_mm_zscore,
    (
      t0.bill_depth_mm - AVG(t0.bill_depth_mm) OVER (PARTITION BY t0.species ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
    )

In [99]:
str(ibis.to_sql(expr)).count("\n")

48